<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ELMo_on_IEMOCAP_for_ERC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
%tensorflow_version 1.x

**This is for ELMo**

In [64]:
!pip install tensorflow-hub

In [65]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [66]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

**This is for Preprocessing and model**

In [67]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:
import pickle
import pandas as pd
import numpy as np
import keras
import itertools
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Input
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical  

from keras.layers import Layer
import keras.backend as K

In [69]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [70]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [71]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [72]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])

In [73]:
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [74]:
def create_batches(data):
  count=0
  last=0
  temp=[]
  batches=[]
  for i in data:
    count=count+1
    temp.append(i)
    if count%100==0:
      count=0
      last=last+1
      batches.append(temp)
      temp=[]
  batches.append(temp)
  return batches

In [75]:
list_train = create_batches(processed_data)
list_test = create_batches(test_processed_data)

In [76]:
#Use the Pre-Embedded pickle files if available. This cell will take a lot of time.

elmo_train = [elmo_vectors(x) for x in list_train]
elmo_test = [elmo_vectors(x) for x in list_test]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [77]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)
elmo_test_new = np.concatenate(elmo_test, axis = 0)

In [78]:
Y=to_categorical(processed_label, num_classes=6)
test_Y=to_categorical(test_processed_label, num_classes=6)

In [84]:
# save elmo_train_new
pickle_out = open("elmo_train_newesttrain.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_newesttest.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()

In [ ]:

# # load elmo_train_new
# pickle_in = open("elmo_train_03032019.pickle", "rb")
# elmo_train_new = pickle.load(pickle_in)

# # load elmo_train_new
# pickle_in = open("elmo_test_03032019.pickle", "rb")
# elmo_test_new = pickle.load(pickle_in)

In [80]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [85]:
inputs=Input((elmo_train_new.shape[1],))
att_in=LSTM(200,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(inputs)
att_out=attention()(att_in)
outputs=Dense(6,activation='softmax')(att_out)
model=Model(inputs,outputs)
model.summary()

ValueError: ignored

In [82]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.fit(elmo_train_new, Y, epochs = 20,  verbose = 1)
score,acc=model.evaluate(elmo_test_new,test_Y,verbose=2)
print("Accuracy : "+ str(acc))

Epoch 1/20
4699/4699 [==============================] - 151s 32ms/step - loss: 1.7462 - accuracy: 0.2460
Epoch 2/20
4699/4699 [==============================] - 147s 31ms/step - loss: 1.7262 - accuracy: 0.2507
Epoch 3/20
4699/4699 [==============================] - 147s 31ms/step - loss: 1.7260 - accuracy: 0.2564
Epoch 4/20
4699/4699 [==============================] - 149s 32ms/step - loss: 1.7237 - accuracy: 0.2564
Epoch 5/20
4699/4699 [==============================] - 147s 31ms/step - loss: 1.7240 - accuracy: 0.2581
Epoch 6/20
4699/4699 [==============================] - 148s 32ms/step - loss: 1.7239 - accuracy: 0.2564
Epoch 7/20
4699/4699 [==============================] - 147s 31ms/step - loss: 1.7235 - accuracy: 0.2547
Epoch 8/20
4699/4699 [==============================] - 148s 32ms/step - loss: 1.7234 - accuracy: 0.2569
Epoch 9/20
4699/4699 [==============================] - 149s 32ms/step - loss: 1.7238 - accuracy: 0.2579
Epoch 10/20
4699/4699 [==============================] 

KeyboardInterrupt: ignored

In [83]:
score,acc=model.evaluate(elmo_test_new,test_Y,verbose=2)
print("Accuracy : "+ str(acc))

Accuracy : 0.23475046455860138
